# Track 2: Key Point Matching [Bert-Base]


### Settings & Initialization

⋅ Install tools and dependencies. 

⋅ Set Environemnt Variables. 

⋅ Define Constants.

In [8]:
!pip install transformers
!pip install optuna

In [9]:
from transformers import DataCollatorWithPadding,TrainingArguments,default_data_collator,EvalPrediction,AutoModelForSequenceClassification,AutoTokenizer,Trainer
import pandas as pd
import os
import numpy as np
import evaluate
import pandas as pd
from sklearn.metrics import average_precision_score


In [10]:
os.environ['TOKENIZERS_PARALLELISM'] = 'true'
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
accuracy = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load("f1")
ID2LABEL = {0: "NEGATIVE", 1: "POSITIVE"}
LABEL2ID = {"NEGATIVE": 0, "POSITIVE": 1}
GOLD_DATA_DIR = './../kpm_data/'

loading configuration file config.json from cache at /Users/riccardoamadio/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/riccardoamadio/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /Users/riccardoamadio/.cache/huggin

### Preprocessing & PostProcessing Methods
 

⋅ Load Raw data by using the protocol defined on paper ArgMining KPA 2021 Shared task.

⋅ Tokenize function to apply before training by using pre-trained tokenizer *distilbert-base-uncased*

⋅ Merge labels,topic and arguments in the same dataframe

⋅ Data Cleaning


In [11]:
def load_kpm_data(subset, submitted_kp_file=None,nrows=None):
    print("\nֿ** loading task data:")
    arguments_file = os.path.join(GOLD_DATA_DIR, f"arguments_{subset}.csv")
    if not submitted_kp_file:
        key_points_file = os.path.join(GOLD_DATA_DIR, f"key_points_{subset}.csv")
    else:
        key_points_file=submitted_kp_file
    labels_file = os.path.join(GOLD_DATA_DIR, f"labels_{subset}.csv")
    arguments_df = pd.read_csv(arguments_file,nrows=nrows)
    key_points_df = pd.read_csv(key_points_file,nrows=nrows)
    labels_file_df = pd.read_csv(labels_file,nrows=nrows)

    return arguments_df, key_points_df, labels_file_df

def preprocess_function(examples):
    token = tokenizer(examples["argument"]+examples["topic"],examples["key_point"],
     add_special_tokens = True,max_length = 80,padding = "max_length")
    token["labels"] = examples["label"]
    return token

def merge_df(arg_df, kp_df, labels_df):
    labels_df=labels_df.merge(kp_df,on="key_point_id",how="left")
    labels_df=labels_df.merge(arg_df,on='arg_id',how="left")
    labels_df.drop(["stance_x", "topic_x"], axis=1, inplace=True)
    labels_df=labels_df.rename(columns={"stance_y":"stance", "topic_y":"topic"})
    return labels_df

def remove_nan(labels_df):
    labels_df = labels_df[labels_df['argument'].notna()]
    labels_df = labels_df[labels_df['topic'].notna()]
    labels_df = labels_df[labels_df['stance'].notna()]
    return labels_df


### Grid Search

⋅ Set configuration of hyperparameter search with [optuna](https://www.example.com)

⋅ Initialize model for the grid search

⋅ Retrieve evaluations metrics to select best model

In [12]:
def optuna_hp_space(trial):
    return {
        "weight_decay": trial.suggest_float("weight_decay", 0.0, 0.1),
        "adam_epsilon": trial.suggest_float("adam_epsilon", 1e-8, 1e-6, log=True),
        "num_train_epochs": trial.suggest_categorical("num_train_epochs", [2,4,8]),
        "batch_size": trial.suggest_categorical("batch_size", [16, 32, 64, 128]),
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 1e-3, log=True),
    }

def model_init(trial):
    return AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",num_labels=2, id2label=ID2LABEL, label2id=LABEL2ID)

def compute_objective(metrics):
    return metrics["eval_loss"]

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions[:, 0]
    predictions = np.where(predictions < 0.5, 0, 1)
    _precision = precision_metric.compute(predictions=predictions, references=labels)
    _map = average_precision_score(labels, predictions)
    _recall = recall_metric.compute(predictions=predictions, references=labels)
    _f1 = f1_metric.compute(predictions=predictions, references=labels)
    _accuracy= accuracy.compute(predictions=predictions, references=labels)
    return {
                "precision": _precision,
                "recall": _recall,
                "f1": _f1,
                "accuracy": _accuracy,
                "map": _map,
            }

In [13]:
if __name__ == "__main__":
    
    
    model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",num_labels=2, id2label=ID2LABEL, label2id=LABEL2ID)
    
    # Merge Labels with arguments and key_points for training dataset
    arg_df_dev, kp_df_dev, labels_df_dev = load_kpm_data( subset="train")
    labels_df_dev=merge_df(arg_df_dev, kp_df_dev, labels_df_dev)
    # Merge Labels with arguments and key_points for test dataset
    arg_df_test, kp_df_test, labels_df_test = load_kpm_data(subset="dev")
    labels_df_test=merge_df(arg_df_test, kp_df_test, labels_df_test)
    # labels_df_test=labels_df_test.merge(kp_df_test,on="key_point_id",how="left")
    # labels_df_test=labels_df_test.merge(arg_df_test,on='arg_id',how="left")
    # labels_df_test.drop(["stance_x", "topic_x"], axis=1, inplace=True)
    # labels_df_test=labels_df_test.rename(columns={"stance_y":"stance", "topic_y":"topic"})
    #encode each sentence and append to dictionary
    # filter where labels are not nan
    # labels_df_dev = labels_df_dev[labels_df_dev['argument'].notna()]
    # labels_df_dev = labels_df_dev[labels_df_dev['topic'].notna()]
    # labels_df_dev = labels_df_dev[labels_df_dev['stance'].notna()]
    labels_df_dev = remove_nan(labels_df_dev)
    labels_df_test = remove_nan(labels_df_test)
    # # labels_df_test = labels_df_test[labels_df_test['argument'].notna()]
    # # labels_df_test = labels_df_test[labels_df_test['topic'].notna()]
    # # labels_df_test = labels_df_test[labels_df_test['stance'].notna()]
    result_train = labels_df_dev.apply(preprocess_function,axis=1)    
    result_test = labels_df_test.apply(preprocess_function,axis=1)
  


    training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=result_train,
    eval_dataset=result_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)
#     trainer = Trainer(
#     model=None,
#     args=training_args,
#     train_dataset=result_train,
#     eval_dataset=result_test,
#     compute_metrics=compute_metrics,
#     tokenizer=tokenizer,
#     model_init=model_init,
#     data_collator=data_collator,
# )
#     train_result = trainer.hyperparameter_search(
#     direction="maximize",
#     backend="optuna",
#     hp_space=optuna_hp_space,
#     n_trials=20,
#     compute_objective=compute_objective,
# )
    train_result = trainer.train()
    metrics = train_result.metrics
    max_train_samples = (len(result_train))
    metrics["train_samples"] = min(max_train_samples, len(result_train))
    trainer.log_metrics("train", metrics)
    

loading configuration file config.json from cache at /Users/riccardoamadio/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /Users/riccardoamadio/.cache/huggingface/hub/models--distilbert-base-uncased/sn


ֿ** loading task data:

ֿ** loading task data:


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/opt/homebrew/anaconda3/envs/key_points/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20635
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 5160
  Number of trainable parameters = 66955010

You're using a DistilBertTokenizerFast tokenizer. Please note that wit

{'loss': 0.4453, 'learning_rate': 9.031007751937986e-06, 'epoch': 0.39}























































































































































































































































































































































































































































































































                                                  

 99%|█████████▉| 250/252 [14:48<00:01,  1.59it/s]  


...

{'loss': 0.2096, 'learning_rate': 3.1007751937984497e-07, 'epoch': 3.88}


































































































































































***** Running Evaluation *****
  Num examples = 3458
  Batch size = 16














































































































































































































































































































































































































































Trainer is attempting to log a value of "{'precision': 0.09111029140538547}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.3346883468834688}" of type <class 'dict'> for key "

{'eval_loss': 0.4557594358921051, 'eval_precision': {'precision': 0.09111029140538547}, 'eval_recall': {'recall': 0.3346883468834688}, 'eval_f1': {'f1': 0.14322992171643956}, 'eval_accuracy': {'accuracy': 0.1454598033545402}, 'eval_map': 0.17248314217255, 'eval_runtime': 39.6321, 'eval_samples_per_second': 87.253, 'eval_steps_per_second': 5.475, 'epoch': 4.0}


Model weights saved in my_awesome_model/checkpoint-5160/pytorch_model.bin
tokenizer config file saved in my_awesome_model/checkpoint-5160/tokenizer_config.json
Special tokens file saved in my_awesome_model/checkpoint-5160/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from my_awesome_model/checkpoint-1290 (score: 0.35746097564697266).

                                                     

100%|██████████| 5160/5160 [11:27:00<00:00,  7.99s/it]

{'train_runtime': 41220.3305, 'train_samples_per_second': 2.002, 'train_steps_per_second': 0.125, 'train_loss': 0.2690989509109379, 'epoch': 4.0}
***** train metrics *****
  epoch                    =         4.0
  train_loss               =      0.2691
  train_runtime            = 11:27:00.33
  train_samples            =       20635
  train_samples_per_second =       2.002
  train_steps_per_second   =       0.125
